In [ ]:
from pathlib import Path
from importlib import reload
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as lines
from astropy.table import Table

import mpl_scatter_density

from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask
from importlib import reload
import utils
reload(utils)

In [ ]:
survey = "south" # choose from `north` or `south
rongpu_path = Path("/global/cscratch1/sd/rongpu/target/catalogs/dr9.0/0.49.0")
my_path = Path("/global/cscratch1/sd/bid13/LRG")

In [ ]:
bgs_path = my_path / ("BGS_SV_" + survey + ".fits")

In [ ]:
bgs_sv_cat = Table.read(bgs_path) # read in the SV data

In [ ]:
bgs_bright_mask = ((bgs_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT_"+survey.upper()))>0)
bgs_faint_mask = ((bgs_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_"+survey.upper()))>0)
lrg_opt_mask = ((bgs_sv_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG_OPT_"+survey.upper()))>0)
lrg_ir_mask = ((bgs_sv_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG_IR_"+survey.upper()))>0)

In [ ]:
print("As percentage of BGS")
print(f"LRG_OPT AND BGS_BRIGHT: {(bgs_bright_mask & lrg_opt_mask).sum()*100/bgs_bright_mask.sum()}")
print(f"LRG_IR AND BGS_BRIGHT: {(bgs_bright_mask & lrg_ir_mask).sum()*100/bgs_bright_mask.sum()}")
print(f"LRG_OPT AND BGS_FAINT: {(bgs_faint_mask & lrg_opt_mask).sum()*100/bgs_faint_mask.sum()}")
print(f"LRG_IR AND BGS_FAINT: {(bgs_faint_mask & lrg_ir_mask).sum()*100/bgs_faint_mask.sum()}")

In [ ]:
# lrg_path = rongpu_path / ("dr9_sv1_lrg_"+ survey+ "_0.49.0_basic.fits")
lrg_path = my_path / ("LRG_SV_" + survey + ".fits")
lrg_sv_cat = Table.read(lrg_path) # read in the SV data

In [ ]:
bgs_bright_mask = ((lrg_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT_"+survey.upper()))>0)
bgs_faint_mask = ((lrg_sv_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_"+survey.upper()))>0)
lrg_opt_mask = ((lrg_sv_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG_OPT_"+survey.upper()))>0)
lrg_ir_mask = ((lrg_sv_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG_IR_"+survey.upper()))>0)

In [ ]:
print("As percentage of LRG")
print(f"LRG_OPT AND BGS_BRIGHT: {(bgs_bright_mask & lrg_opt_mask).sum()*100/lrg_opt_mask.sum()}")
print(f"LRG_IR AND BGS_BRIGHT: {(bgs_bright_mask & lrg_ir_mask).sum()*100/lrg_ir_mask.sum()}")
print(f"LRG_OPT AND BGS_FAINT: {(bgs_faint_mask & lrg_opt_mask).sum()*100/lrg_opt_mask.sum()}")
print(f"LRG_IR AND BGS_FAINT: {(bgs_faint_mask & lrg_ir_mask).sum()*100/lrg_ir_mask.sum()}")

In [ ]:
utils.ts_plot(lrg_sv_cat[lrg_ir_mask], lrg_sv_cat[bgs_faint_mask & lrg_ir_mask])

In [ ]:
z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3/LRG/alltiles_LRGzinfo.fits")

z_cat = z_cat[(z_cat["FIBERSTATUS"]==0) & (z_cat["ZWARN"]==0)]
z_cat = z_cat[z_cat["PHOTSYS"]==survey.upper()[0]]
z_cat = z_cat[z_cat["SPECTYPE"]=="GALAXY"]
z_cat = z_cat[z_cat["TARGETS"] == "QSO+LRG"]

names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
z_cat = z_cat[names].to_pandas()
z_cat = z_cat[z_cat.duplicated(subset="TARGETID")][["TARGETID", "Z"]]
z_cat = z_cat.merge(lrg_sv_cat.to_pandas(), how="inner", on= "TARGETID")
z_cat = Table.from_pandas(z_cat)

In [ ]:
bgs_bright_mask = ((z_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT_"+survey.upper()))>0)
bgs_faint_mask = ((z_cat["SV1_BGS_TARGET"] & bgs_mask.mask("BGS_FAINT_"+survey.upper()))>0)
lrg_opt_mask = ((z_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG_OPT_"+survey.upper()))>0)
lrg_ir_mask = ((z_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG_IR_"+survey.upper()))>0)

In [ ]:
utils.ts_plot(z_cat[lrg_ir_mask], z_cat[bgs_faint_mask & lrg_ir_mask], cat_frac=1, extra_cat_frac=1)

In [ ]:
# load_randoms to calculate area
random_path = Path("/global/cfs/cdirs/desi/target/catalogs/dr9/0.47.0/randoms/resolve/randoms-1-13.fits")
randoms = Table.read(random_path)
randoms = randoms[utils.not_in_mask("LRG", randoms) & (randoms["PHOTSYS"]==survey.upper()[0])]

In [ ]:
area = len(randoms)/2500 #sq degree
del randoms

In [ ]:
utils.plot_comoving_density(z_cat["Z"], area,zmin=0.0, zmax=1.5, axis=[0.0, 1.25, 0, 0.01] )

In [ ]:
utils.plot_comoving_density(lrg_sv_cat[lrg_ir_mask]["zphot"], area,zmin=0.1, zmax=1.5, axis=[0.1, 1.5, 0, 0.85], show=False )
utils.plot_comoving_density(lrg_sv_cat[(~bgs_bright_mask) & lrg_ir_mask]["zphot"], area,zmin=0.1, zmax=1.5, axis=[0.1, 1.5, 0, 0.85], show=False)

In [ ]:
utils.plot_comoving_density(lrg_sv_cat[lrg_opt_mask]["zphot"], area,zmin=0.1, zmax=1.5, axis=[0.1, 1.5, 0, 0.85], show=False )
utils.plot_comoving_density(lrg_sv_cat[(~bgs_bright_mask) & lrg_opt_mask]["zphot"], area,zmin=0.1, zmax=1.5, axis=[0.1, 1.5, 0, 0.85], show=False)